In [3]:
import pandas as pd
import time
from datetime import date
from datetime import time
import matplotlib.pyplot as plt
import re

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import stem
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ironhack")
from geopy.util import get_version
get_version()

[nltk_data] Downloading package stopwords to /home/laura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/laura/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'1.18.1'

Importamos el archivo que hemos limpiado y enriquecido:

In [4]:
data_m = pd.read_csv('./UFO_modify1.csv')
data_m.head()

,Unnamed: 0,datetime,city,state,country,shape,duration (seconds),comments,date posted,latitude,longitude,date,year,time
0,0,1949-10-10 20:30:00,San Marcos,TX,US,cylinder,2700.0,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,1949-10-10,1949.0,20:30:00
1,1,1949-10-10 21:00:00,Lackland Afb,TX,timeout,light,7200.0,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949-10-10,1949.0,21:00:00
2,2,1955-10-10 17:00:00,Chester (Uk/England),NaN,GB,circle,20.0,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955-10-10,1955.0,17:00:00
3,3,1956-10-10 21:00:00,Edna,TX,US,circle,20.0,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,1956-10-10,1956.0,21:00:00
4,4,1960-10-10 20:00:00,Kaneohe,HI,US,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,1960-10-10,1960.0,20:00:00


Vemos si nuestra columna 'country' ha quedado como necesitamos:

In [5]:
data_m.country.value_counts()

US             70105
timeout        10526
CA              3243
GB              2038
AU               578
DE               111
USA               14
UK                 2
España             2
Brasil             2
México             1
Ghana              1
Philippines        1
India              1
Malaysia           1
Slovenija          1
Sverige            1
Portugal           1
Name: country, dtype: int64

Eliminaremos las filas cuyo pais es 'timeout' (la API consultada nos genera un error), y convertimos un par de paises al codigo de 2 letras:

In [6]:
#vamos a eliminar las filas cuyo pais es 'timeout':
data_m = data_m[data_m['country'] != 'timeout']

#homogeneizamos los valores de la columna 'country':

data_m['country'] = data_m['country'].replace(['USA'], 'US')
data_m['country'] = data_m['country'].replace(['México'], 'MX')

data_m.country.unique()

array(['US', 'GB', 'CA', 'AU', 'Portugal', 'DE', 'Ghana', 'India',
       'España', 'UK', 'Sverige', 'Malaysia', 'Brasil', 'Slovenija', 'MX',
       'Philippines'], dtype=object)

## 3. Análisis NLP

In [ ]:
Antes de hacer el analisis de NLP, debemos eliminar las filas con valores nulos:

In [8]:
data_clean = data_m.dropna(axis=0, how='any', inplace=False)

#cambiamos los tipos de 'year' y 'duration(seconds)' a números enteros:
data_clean['year'] = data_clean['year'].astype(int)
data_clean['duration (seconds)'] = data_clean['duration (seconds)'].astype(int)

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Antes de seguir con nuestro análisis NLP, guardamos la data limpia en otro .csv, ya que mas adelate lo necesitaremos para analizar los datos con Tableau.

In [ ]:
#data_clean.to_csv("UFO_modify2.csv")

Empezamos con el análisis NLP:
 - seleccionamos las ultimas 50.000 filas
 - limpiamos los caracteres especiales y los números.
 - tokenizamos
 - stemming y lemmatizazion
 - limpiamos stop-words
 - hacemos zip entre las palabras y su frecuencia de aparición; seleccionamos las 100 más repetidas.

In [9]:
#seleccionamos las ultimas 50000 filas:

texto = list(data_clean["comments"])[:-50000]

#limpiamos los carateres especiales y los numeros:

def clean_up(s):
    pattern = '[^a-z]+'
    x = re.sub(pattern, ' ', str(s).lower())
    return x.split('http')[0]

In [11]:
def tokenize(s):
    return word_tokenize(s)

texto_s = clean_up(texto)

#print (type(texto_s))
#stemming and lemmatizacion:

def stem_and_lemmatize(l):
    lemmatizer = WordNetLemmatizer()
    stemmer = stem.PorterStemmer()
    stem_phrase = [stemmer.stem(x) for x in l]
    stem_and_lemmatizer_phrase = [lemmatizer.lemmatize(x) for x in stem_phrase]
    return stem_and_lemmatizer_phrase

palabras = tokenize(texto_s)

texto_ssl = stem_and_lemmatize(palabras)

In [12]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(texto_ssl)

bag_of_words = list(vectorizer.get_feature_names())

freq_matrix = X.toarray()

freq_matrix_sum = list(freq_matrix.sum(axis=0))

zipall = list(zip(bag_of_words, freq_matrix_sum))

sorted_zipall = sorted(zipall, key=lambda x: x[1], reverse=True)

#print(sorted_zipall)


#Se deben limpiar algunas stop-words
clean_vocab = list()

stopW = stopwords.words('english')

clean_vocab.append([tuple for tuple in sorted_zipall if tuple[0] not in stopW])
most_common = clean_vocab[0:100]
most_common

[[('light', 1581),
  ('object', 879),
  ('sky', 736),
  ('wa', 581),
  ('bright', 539),
  ('move', 510),
  ('shape', 500),
  ('white', 389),
  ('red', 334),
  ('orang', 323),
  ('saw', 284),
  ('seen', 268),
  ('craft', 262),
  ('fli', 252),
  ('veri', 252),
  ('ufo', 249),
  ('hover', 245),
  ('like', 230),
  ('star', 224),
  ('larg', 207),
  ('night', 186),
  ('green', 181),
  ('quot', 180),
  ('sight', 174),
  ('flash', 168),
  ('north', 167),
  ('two', 166),
  ('blue', 162),
  ('one', 161),
  ('triangl', 160),
  ('look', 157),
  ('color', 150),
  ('abov', 144),
  ('appear', 142),
  ('fast', 132),
  ('high', 132),
  ('south', 131),
  ('disappear', 129),
  ('low', 128),
  ('speed', 127),
  ('ball', 126),
  ('travel', 126),
  ('format', 125),
  ('sound', 125),
  ('glow', 120),
  ('sphere', 120),
  ('three', 120),
  ('west', 117),
  ('across', 115),
  ('firebal', 115),
  ('pd', 115),
  ('silent', 105),
  ('east', 104),
  ('nuforc', 103),
  ('around', 102),
  ('note', 102),
  ('triangul

## 4. Visualización: 

### WORDCLOUD_UFO: 

con esta lista de palabras, crearemos una WordCloud con python, tal y como podemos ver en el archivo 'WordCloud_UFO.ipynb'.

### GRAPHEXT:

He utilizado Graphext y su herremienta Traktor, para analizar si la publicacion de Loeb y Bialy afecto a la opinion pública, analizando el hashtag #UFO en torno a las fechas de publicación.

Tambien he represerntado los Emojis más representados en esos twitts.

### TABLEAU:

Tableau me ha ayudado a analizar y representar los datos del  Dataset:

- nº de avistamientos mundial por década
- nº de avistamientos por pais y década
- distribucion en base a la forma observada
- formas mas observadas segun el pais
- horas del dia en el que se han observado más avistamientos

### FLOURISH:

Para concluir el proyecto, he realizado un mapa temporal utilizando los datos de geolocalizacion del data, en el que podemos observar los avistamientos segun su localizacion, y la fecha en la que ocurrieron.